In [1]:
from ProbRobNLP.dialogue import Dialogue
from ProbRobNLP import logic
from itertools import product
import maxentropy
from collections import namedtuple

import numpy as np

In [14]:
import torch
from torch.autograd import Variable
from torch.nn import functional as f
xdata = Variable(torch.Tensor([[12.0], [11.0], [5.0], [4.0]]))
ydata = Variable(torch.Tensor([[92.0], [82.0], [52.0], [32.0]]))
class logistic_regression(torch.nn.Module):
     def __init__(self):
        super(logistic_regression, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
     def forward(self, y):
        ypred = f.sigmoid(self.linear(y))
        return ypred
models = logistic_regression()
criterions = torch.nn.BCELoss(size_average=True)
optimizers = torch.optim.SGD(models.parameters(), lr=0.01)
for epoch in range(20):
    models.train()
    optimizers.zero_grad()
    # Forward pass
    ypred = models(xdata)
    # Compute Loss
    losses = criterions(ypred, ydata)
    # Backward pass
    losses.backward()
    optimizers.step()
x = Variable(torch.Tensor([[6.0]]))
ypred = models(x)
print("predicted y value: ", ypred.data[0][0])

predicted y value:  tensor(1.)


/Users/mattiasappelgren/miniconda3/envs/probrobnlp/lib/python3.9/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/Users/mattiasappelgren/miniconda3/envs/probrobnlp/lib/python3.9/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [2]:
def score_candidate(unresolved, candidate, current_state):

    print(f'{unresolved = }', f'{candidate = }')
    
    print(f'{unresolved.features = }', f'{candidate.features = }')
    
    print('dobj', candidate.features['dobj'], 'nr agree', candidate.features['number'] == unresolved.features['number'], 
          )


def resolve_reference(self):
    unresolved = [r for r in self.current_state.ref if isinstance(r, logic.Variable)]
    if len(unresolved) > 0:

        print('unresolved', unresolved)

        for r in unresolved:
            candidates = self.get_candidates(r)
            for c in candidates:
                score_candidate(r, c, self.current_state)
            

Dialogue.resolve_reference = resolve_reference 

In [3]:
d = Dialogue()
d.update('put a table in the centre')
d.update('put a cube on it')

unresolved [X8]
[]
[e2, x1, e1, X8, x0, e0, Vector3D(0, 0, 0), e3][put(e0, x0, x0), table(x0), in(e1, x0, Vector3D(0, 0, 0)), cube(x1), put(e2, x1, x1), on(e3, x1, X8)] h: {} f: {}
[e2, x1, e1, X8, x0, e0, Vector3D(0, 0, 0), e3]
[x1, x0]
x1 True
x0 True
unresolved = X8 candidate = x1
unresolved.features = [{'proper_noun': False, 'pronoun': False, 'dobj': False, 'iobj': False, 'words': ['it'], 'positions': [[4]], 'number': 1}] candidate.features = [{'proper_noun': False, 'pronoun': False, 'dobj': False, 'iobj': False, 'words': ['a'], 'positions': [[1]], 'number': 1}]
[{'proper_noun': False, 'pronoun': False, 'dobj': False, 'iobj': False, 'words': ['it'], 'positions': [[4]], 'number': 1}]
unresolved = X8 candidate = x0
unresolved.features = [{'proper_noun': False, 'pronoun': False, 'dobj': False, 'iobj': False, 'words': ['it'], 'positions': [[4]], 'number': 1}] candidate.features = [{'proper_noun': False, 'pronoun': False, 'dobj': False, 'iobj': False, 'words': ['a'], 'positions': [[1]],

In [13]:


class CorefMaxentModel:
    
    def __init__(self, features, functions):
        self.features = features
        self.feature_vector = namedtuple('feature_vector', self.features.keys())
        self.feature_names = [[v for v in values] for key, values in self.features.items()]
        #'time_x':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],'time_y':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        self.samplespace = [self.feature_vector(*v) for v in product(*self.feature_names)]
        def f(x):
            return int(x in self.samplespace)
        self.functions = functions + [f]
        
        self.model = maxentropy.Model(self.functions, self.samplespace, vectorized=False)
        
    def fit_model(self, K):
        self.model.fit(K + [1])
        self.probs = {sample: p for sample, p in zip(self.samplespace, self.model.probdist())}
        
    def evaluate(self, sample):
        vector_positive = self.feature_vector(*(sample + [True]))
        p_positive = self.probs[vector_positive]
        
        return p_positive, vector_positive
    
    def p(self, samples):
        ps, samples = zip(*[self.evaluate(sample) for sample in samples])
        ps = np.array(ps)

        p_vector = ps/np.sum(ps)
        return {sample: p for sample, p in zip(samples, p_vector)}
    
    def show(self):
        for sample, p in zip(self.samplespace, self.model.probdist()):
            print(sample, p)
            
            
features = {'dep_x':('dobj', 'iobj', 'pobj', 'subj'), 'time_diff': [0, 1, 2, 3, 4], 'number_x':('s', 'p'), 
            'dep_y':('dobj', 'iobj', 'pobj', 'subj'), 'number_y':('s', 'p'),  'ref':[True, False]}

features_lap_leas = {'rec': [True, False], 'DEP': ['subj', 'obj', 'indobj'], 
                     'Exist': [True, False], 'nonadv':[True, False], 'headN':[True, False]}

def recency(x):
    return int(x.rec)

def subj(x):
    return int(x.DEP == 'subj')

def obj(x):
    return int(x.DEP == 'obj')

def indobj(x):
    return int(x.DEP == 'indobj')

def exist(x):
    return int(x.Exist)

def nonadv(x):
    return int(x.nonadv)

def headn(x):
    return int(x.headN)


f_lap_leas = [recency, subj, obj, indobj, exist, nonadv, headn]
K_lap_leas = [0.9, 0.8, 0.6, 0.55, 0.6, 0.7, 0.85]

model = CorefMaxentModel(features_lap_leas, f_lap_leas)
model.fit_model(K_lap_leas)



def f0(x):
    return int(x.dep_x == 'dobj' and x.ref)

def number_agreement(x):
    return int(x.number_x == x.number_y and x.ref)

def recency(x):
    return int(abs(x.time_diff - 1))

f = [f0, number_agreement, recency]

K = [0.8, 0.8, 1]
model = CorefMaxentModel(features, f)
model.fit_model(K)

# ps = model.p([['subj', 0, 's', 'subj', 's'],
#          ['subj', 1, 's', 'subj', 's'],
#          ['subj', 2, 's', 'subj', 's'],
#          ['subj', 3, 's', 'subj', 's']])
# for s, p in ps.items():
#     print(s, p)

# print()
# for s, p in model.p([
#         ['dobj', 1, 's', 'subj', 's'],
#         ['subj', 1, 's', 'subj', 's'],
#         ['pobj', 1, 's', 'subj', 's']
#         ]).items():
#     print(s,p)


    
# model.show()
# model = maxentropy.Model(f, samplespace, vectorized=False)
# model.verbose = True

# model.fit(K)

DivergenceError: "dual is below the threshold 'mindual' and may be diverging to -inf.  Fix the constraints or lower the threshold!"

In [65]:
model.probs

{feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=0, number_y='s', ref=True): 0.0038518286070732067,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=0, number_y='s', ref=False): 4.0752805462287504e-05,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=0, number_y='p', ref=True): 0.0006862350350000322,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=0, number_y='p', ref=False): 4.0752805462287504e-05,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=1, number_y='s', ref=True): 0.0023010360020707184,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=1, number_y='s', ref=False): 2.4345234983173675e-05,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=1, number_y='p', ref=True): 0.0004099485419776162,
 feature_vector(dep_x='dobj', time_x=0, number_x='s', dep_y='dobj', time_y=1, number_y='p', ref=False): 2.434523

In [23]:
list(sample_space)

[('dobj', 's', 0, 'dobj', 's', 0, True),
 ('dobj', 's', 0, 'dobj', 's', 0, False),
 ('dobj', 's', 0, 'dobj', 's', 1, True),
 ('dobj', 's', 0, 'dobj', 's', 1, False),
 ('dobj', 's', 0, 'dobj', 's', 2, True),
 ('dobj', 's', 0, 'dobj', 's', 2, False),
 ('dobj', 's', 0, 'dobj', 's', 3, True),
 ('dobj', 's', 0, 'dobj', 's', 3, False),
 ('dobj', 's', 0, 'dobj', 's', 4, True),
 ('dobj', 's', 0, 'dobj', 's', 4, False),
 ('dobj', 's', 0, 'dobj', 's', 5, True),
 ('dobj', 's', 0, 'dobj', 's', 5, False),
 ('dobj', 's', 0, 'dobj', 's', 6, True),
 ('dobj', 's', 0, 'dobj', 's', 6, False),
 ('dobj', 's', 0, 'dobj', 's', 7, True),
 ('dobj', 's', 0, 'dobj', 's', 7, False),
 ('dobj', 's', 0, 'dobj', 's', 8, True),
 ('dobj', 's', 0, 'dobj', 's', 8, False),
 ('dobj', 's', 0, 'dobj', 's', 9, True),
 ('dobj', 's', 0, 'dobj', 's', 9, False),
 ('dobj', 's', 0, 'dobj', 's', 10, True),
 ('dobj', 's', 0, 'dobj', 's', 10, False),
 ('dobj', 's', 0, 'dobj', 'p', 0, True),
 ('dobj', 's', 0, 'dobj', 'p', 0, False),
 (